In [1]:
import cntk as C
from PIL import Image
import os
import numpy as np
import urllib
from scipy.misc import imsave

try:
    from urllib.request import urlretrieve, urlopen
except ImportError:
    from urllib import urlretrieve, urlopen

try:
    C.device.try_set_default_device(C.device.gpu(0))
except:
    print("GPU unavailable. Using CPU instead.")

In [2]:
#filename - relative path of image being processed
#outfile - relative path of the image which will be saved

def evaluate(model, img):
    output_dims = 64

    #upscaling coefficient
    coef = 2

    #at each step, we will evaluate subpatch (x : x + range_x, y : y + range_y) of original image
    #patch by patch, we will resolve the whole image
    range_x = output_dims // coef
    range_y = output_dims // coef

    #how many bounding pixels from resulting patch should be excluded?
    #this is important because boundaries tend to be predicted less accurately
    offset = output_dims // 10

    #after we evaluate a subpatch, how much we move down/right to get the next one
    #we subtract offset to cover those pixels which were boundary in the previous subpatch
    step_x = range_x - offset
    step_y = range_y - offset

    img = img.resize((coef * img.width, coef * img.height), Image.BICUBIC)

    result = np.zeros((img.height, img.width, 3))
    range_x = output_dims
    range_y = output_dims
    step_x = range_x - 2 * offset
    step_y = range_y - 2 * offset
    coef = 1

    rect = np.array(img, dtype = np.float32)

    #if the image is too small for some models to work on it, pad it with zeros
    if(rect.shape[0] < range_y):
        pad = np.zeros((range_y - rect.shape[0], rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 0).astype(dtype = np.float32)

    if(rect.shape[1] < range_x):
        pad = np.zeros((rect.shape[0], range_x - rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 1).astype(dtype = np.float32)

    x = 0
    y = 0
    
    #take subpatch by subpatch and resolve them to get the final image result
    while(y < img.width):
        x = 0
        while(x < img.height):
            rgb_patch = rect[x : x + range_x, y : y + range_y]
            rgb_patch = rgb_patch[..., [2, 1, 0]]
            rgb_patch = np.ascontiguousarray(np.rollaxis(rgb_patch, 2))
            pred = model.predict(rgb_patch.transpose(2,1,0).reshape(1,64,64,3))

            img2 = pred.reshape(64,64,3)

            # make sure img2 is C Contiguous as we just transposed it
            img2 = np.ascontiguousarray(img2)
            #make sure no pixels are outside [0, 255] interval
            for _ in range(2):
                img2 = C.relu(img2).eval()
                img2 = np.ones(img2.shape) * 255.0 - img2

            rgb = img2[..., ::-1]
            patch = rgb.transpose(1, 0, 2)

            #fill in the pixels in the middle of the subpatch
            #don't fill those within offset range to the boundary
            for h in range(coef * x + offset, coef * x + output_dims - offset):
                for w in range(coef * y + offset, coef * y + output_dims - offset):
                    for col in range(0, 3):
                        result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #pad top
            if(x == 0):
                for h in range(offset):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h][w - coef * y][col]

            #pad left
            if(y == 0):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(offset):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w][col]

            #pad bottom
            if(x == img.height - range_x):
                for h in range(coef * img.height - offset, coef * img.height):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #pad right
            if(y == img.width - range_y):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(coef * img.width - offset, coef * img.width):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #reached bottom of image
            if(x == img.height - range_x):
                break
            #next step by x, we must not go out of bounds
            x = min(x + step_x, img.height - range_x)

        #reached right edge of image
        if(y == img.width - range_x):
            break
        #next step by y, we must not go out of bounds
        y = min(y + step_y, img.width - range_x)

    result = np.ascontiguousarray(result)

    #save result
    return result.astype(np.uint8)

In [3]:
import cv2
import os
dict_of_vid= {
    'vid_3_360.mp4':{'fps':0,'data':[]},
    'vid_2_360.mp4':{'fps':0,'data':[]},
    'vid_1_360.mp4':{'fps':0,'data':[]}
}

In [4]:
for f in dict_of_vid:
    vidcap = cv2.VideoCapture(f)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    dict_of_vid[f]['fps'] = fps
    success,image = vidcap.read()
    while success:
        img = Image.fromarray(image, mode='RGB')
        dict_of_vid[f]['data'].append(img)
        success,image = vidcap.read()

In [5]:
from keras.models import load_model
model = load_model('..//Prediction//data//vihance//Models//vdsr_trained.hdf5')

C:\Users\Aldi\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using CNTK backend


In [6]:
def evaluate_vid(f, fps, data):
    print("processing", f)
    print("len",str(len(data)))
    first = True
    fourcc = None
    out = None
    enhanced = []
    bicubic = []
    bicubic_file = 'bicubic_' + str(f.split('.')[0])
    enhanced_file = 'enhanced_' + str(f.split('.')[0])
    ct = 0
    for d in data:
        print('enhanced',str(ct))
        ct+=1
        enhc = evaluate(model, d)
        bgr_img = np.array(d, dtype = np.uint8)
        rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
        bic_img = Image.fromarray(rgb_img, mode='RGB')
        bic = bic_img.resize((2 * bic_img.width, 2 * bic_img.height), Image.BICUBIC)

        enhanced.append(enhc)
        bicubic.append(bic)
        cv2.imwrite('./enhanced/{}{}.jpg'.format(f.split('.')[0], ct), enhc)
        with open('./bicubic/{}{}.jpg'.format(f.split('.')[0], ct), 'w') as fss:
            bic.save(fss)
        if ct == fps*5:
            break
    print("done enhance")
    try:
        for enhc in enhanced:
            if first:
                h,w,c = enhc.shape
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter('{}.avi'.format(enhanced_file), fourcc, fps, (w, h))
                first = False
            print('write',str(ct))
            ct+=1
            out.write(enhc)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        out.release()
        cv2.destroyAllWindows()
        first = True
        print("DONE WRITING ENHANCED")
        for bic in bicubic:
            if first:
                h,w,c = bic.shape
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter('{}.avi'.format(bicubic_file), fourcc, fps, (w, h))
                first = False
            print('write',str(ct))
            ct+=1
            out.write(bic)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        out.release()
        cv2.destroyAllWindows()
        print("DONE WRITING ENHANCED")
        return (enhanced, bicubic)
    except:
        print("")
        return False

In [ ]:
fails = []
for f in dict_of_vid:
    fails.append(evaluate_vid(f,  dict_of_vid[f]['fps'], dict_of_vid[f]['data']))